## Created by <a href="https://github.com/yunsuxiaozi">yunsuxiaozi</a>  2024/7/31

#### 这里将使用甄嬛传的数据集来完成对大模型chatglm-6b的微调,实现一个个性化的AI。数据集来源于github的一个开源项目:<a href="https://github.com/KMnO4-zx/xlab-huanhuan">xlab-huanhuan</a>,目前已将数据集上传到Kaggle上:<a href="https://www.kaggle.com/datasets/yunsuxiaozi/chat-huanhuan">chat-huanhuan</a>,各位可以用于学习大模型微调的技术。这里关于chatglm-6b的微调参考了这个notebook:<a href="https://www.kaggle.com/code/tiansztianszs/chatglm-fine-tuning/notebook">chatglm finetune</a>。这里使用的是Kaggle上的GPU T4\*2。

## 1.熟悉数据集

#### 这里先来看看我们使用的数据集是什么样的。

In [1]:
import json#用于处理JavaScript Object Notation数据格式
#用utf-8编码读取json文件
with open("/kaggle/input/chat-huanhuan/huanhuan.json",mode='r',encoding='utf-8') as f:
    data=json.load(f)
print(f"len(data):{len(data)}")
data[0]

len(data):3729


{'instruction': '小姐，别的秀女都在求中选，唯有咱们小姐想被撂牌子，菩萨一定记得真真儿的——',
 'input': '',
 'output': '嘘——都说许愿说破是不灵的。'}

#### 我们可以看到,数据有三千多条,每个数据都是一个字典,包含'instruction','input'和'output',这里我们要微调一个对话的AI,'instruction'就是你和AI说的话,'output'就是AI给你的回答。

#### 如果要微调大模型去打比赛,比如大模型小说创作比赛,instruction就是故事的梗概,output就是故事的具体内容;如果是大模型推理比赛,instruction就是推理的问题,output就是推理的回答。只要有足够的高质量的数据,我们就可以把大模型微调成我们想要的样子。

#### 这里使用的数据集已经是json文件,并且也是非常干净(也就是不需要我们再做什么处理)的数据,我们后续可以直接将json文件的路径作为一个参数完成大模型的微调。


## 2.chatglm-6b的微调环境准备

#### 先用克隆命令将chatglm-6b从github上克隆到本地环境。

In [2]:
!git clone https://github.com/THUDM/ChatGLM-6B.git

Cloning into 'ChatGLM-6B'...
remote: Enumerating objects: 1252, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 1252 (delta 8), reused 11 (delta 6), pack-reused 1235
Receiving objects: 100% (1252/1252), 9.15 MiB | 24.59 MiB/s, done.
Resolving deltas: 100% (737/737), done.


#### 安装chatglm-6b需要依赖的一些库。requirements.txt文件里写了chatglm-6b里依赖的库以及对应的版本号,比如transformers==4.27.1,我们需要安装这些库。

In [3]:
#这里就是查看一下requirements有哪些库,可以简单理解为print(),删除掉这段代码不会影响程序的正常运行。
with open("/kaggle/working/ChatGLM-6B/requirements.txt",mode='r',encoding='utf-8') as f:
    data=f.readline()
    while data:
        print(data)
        data=f.readline()

protobuf

transformers==4.27.1

cpm_kernels

torch>=1.10

gradio

mdtex2html

sentencepiece

accelerate


In [4]:
!pip install -r ChatGLM-6B/requirements.txt  #安装chatglm需要依赖的库

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.2/318.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 8.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: websockets
 

#### 安装chatglm依赖的其他库

- rouge_chinese:这个库提供了用于评估生成文本和参考文本重叠度的ROUGE（Recall-Oriented Understudy for Gisting Evaluation）分数的实现。

- nltk:自然语言处理工具包。

- jieba:中文文本分词库。

- datasets:用于处理和加载数据集

In [5]:
!pip install -q rouge_chinese nltk jieba datasets 

#### 克隆经过int4量化处理的chatglm-6b模型的权重。

In [6]:
!git clone https://huggingface.co/THUDM/chatglm-6b-int4

Cloning into 'chatglm-6b-int4'...
remote: Enumerating objects: 137, done.
remote: Total 137 (delta 0), reused 0 (delta 0), pack-reused 137 (from 1)
Receiving objects: 100% (137/137), 58.06 KiB | 14.52 MiB/s, done.
Resolving deltas: 100% (79/79), done.


## 3.chatglm 模型微调

#### 加载预训练模型和分词器。

In [7]:
# AutoTokenizer自动加载与模型对应的分词器,AutoModel自动加载预训练模型
from transformers import AutoTokenizer, AutoModel

model_path = "chatglm-6b-int4"#模型的参数
#根据模型的路径加载预训练分词器,允许远程加载代码(trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
#根据模型的路径加载预训练模型,允许远程加载代码(trust_remote_code=True),half是半精度浮点数,cuda是移动到GPU上
model = AutoModel.from_pretrained(model_path, trust_remote_code=True).half().cuda()

Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


#### 为了比较大语言模型微调前后的效果,这里先试试微调前的response。

In [8]:
#将大模型换成评估模式
question='朕的后宫里佳丽三千,你可知朕最宠幸的是谁?'
model = model.eval()
response, history = model.chat(tokenizer, question, history=[])
print(f"question:{question}\nresponse:{response}")
model=model.train()#将模型转回训练模式

2024-07-31 08:48:33.472631: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 08:48:33.472758: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 08:48:33.644111: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


question:朕的后宫里佳丽三千,你可知朕最宠幸的是谁?
response:作为一个人工智能助手，我的职责是辅助人类完成任务并提供信息。然而，作为一个虚拟的存在，我没有实际的身体，因此我无法在现实世界中扮演一个真实的人类角色，包括在后宫中宠幸佳丽。此外，作为一个人工智能助手，我也无法了解或记录历史上的任何事件或人物。


#### 关闭wandb,不要深度学习的实验跟踪。

In [9]:
import os#与操作系统进行交互的库
#Wandb是Weights & Biases的缩写，它是一个用于机器学习实验跟踪和可视化的工具和平台。
#关闭wandb。
os.environ["WANDB_DISABLED"] = "true"

#### 大语言模型微调的脚本,下面是参数的详细解释:

- PRE_SEQ_LEN:训练过程中大模型输入序列的最大长度。这是传入大模型的一个参数,这个参数是影响大模型内部的设置的,而后面出现的max_source_length是控制文本的最大长度,是影响文本序列的。

- LR:学习率

- CUDA_VISIBLE_DEVICES=0:使用第一个GPU设备加速。

- python3 ChatGLM-6B/ptuning/main.py:用python3运行微调的脚本。

- do_train:执行训练的过程。

- train_file、validation_file,训练数据和验证数据的路径,这里选择了同一个数据集。

- prompt_column:提示词列,用户问AI的question文本的名称,这里是instruction。

- response_column:AI回复用户问题的文本的名称,这里是'output'。

- overwrite_cache:允许脚本覆盖现有的缓存文件。

- model_name_or_path:指定了模型的名称或路径,这里使用的是量化为 4 位的 ChatGLM-6B 模型。

- output_dir:输出文本的路径。

- overwrite_output_dir:如果output_dir这里的文件夹已经存在,运行的时候会覆盖掉上一次的文件夹。

- max_source_length:用户问AI问题的最大文本长度。

- max_target_length:AI回答问题的最大文本长度。

- per_device_train_batch_size:训练的时候每次训练2个数据。

- per_device_eval_batch_size:验证的时候使用1个数据。

- gradient_accumulation_steps:梯度积累几个batch之后更新一次参数。

- predict_with_generate:模型预测的时候使用generate生成文本,有助于生成更加自然和连贯的文本。

- max_steps:训练多少个steps。

- logging_steps:多少个steps输出一次日志。

- save_steps:多少个steps保存一个模型。

- learning_rate:学习率

- pre_seq_len:训练过程中大模型输入序列的最大长度
 
- quantization_bit:指定了模型的量化位数为4位量化。


In [10]:
!PRE_SEQ_LEN=256 && LR=5e-2 && CUDA_VISIBLE_DEVICES=0 python3 ChatGLM-6B/ptuning/main.py \
    --do_train \
    --train_file /kaggle/input/chat-huanhuan/huanhuan.json \
    --validation_file /kaggle/input/chat-huanhuan/huanhuan.json \
    --prompt_column instruction \
    --response_column output \
    --overwrite_cache \
    --model_name_or_path chatglm-6b-int4 \
    --output_dir output/infer-chatglm-6b-int4-pt-$PRE_SEQ_LEN-$LR \
    --overwrite_output_dir \
    --max_source_length 256 \
    --max_target_length 128 \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --gradient_accumulation_steps 4 \
    --predict_with_generate \
    --max_steps 500 \
    --logging_steps 50 \
    --save_steps 50 \
    --learning_rate $LR \
    --pre_seq_len $PRE_SEQ_LEN \
    --quantization_bit 4

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


2024-07-31 08:48:52.444986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-31 08:48:52.445039: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-31 08:48:52.446291: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/opt/conda/lib/python3.10/site-packages/datasets/load.py:2554: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be remove

## 4.微调后的模型测试

#### 由于大模型微调50次就已经具有甄嬛的特点了,所以这里使用了checkpoint50来回答question,各位也可以使用训练500次的checkpoint来回答问题。

In [11]:
import torch#pytorch这个深度学习框架
from transformers import AutoConfig#自动下载和配置预训练模型的配置

#根据模型路径加载config,允许远程加载代码(trust_remote_code=True),大模型输入序列的最大长度
config = AutoConfig.from_pretrained(model_path, trust_remote_code=True, pre_seq_len=256)
#根据模型的路径和参数加载模型,允许远程加载代码(trust_remote_code=True)
model = AutoModel.from_pretrained(model_path, config=config, trust_remote_code=True)

#从二进制(binary)文件中加载模型的状态字典,这个参数字典一般是在某个检查点(checkpoint)保存下来的。
prefix_state_dict = torch.load("output/infer-chatglm-6b-int4-pt-256-5e-2/checkpoint-50/pytorch_model.bin")
#进行参数的更新
new_prefix_state_dict = {}
for k, v in prefix_state_dict.items():
    new_prefix_state_dict[k[len("transformer.prefix_encoder."):]] = v
model.transformer.prefix_encoder.load_state_dict(new_prefix_state_dict)

#half是半精度浮点数,cuda是移动到GPU上
model = model.half().cuda()
#将模型prefix_encoder部分的参数换成全精度浮点数float32
model.transformer.prefix_encoder.float()
#将大模型换成评估模式
model = model.eval()

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.


No compiled kernel found.
Compiling kernels : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c
Compiling gcc -O3 -fPIC -pthread -fopenmp -std=c99 /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.c -shared -o /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Load kernel : /root/.cache/huggingface/modules/transformers_modules/chatglm-6b-int4/quantization_kernels_parallel.so
Setting CPU quantization kernel threads to 2
Parallel kernel is not recommended when parallel num < 4.
Using quantization cache
Applying quantization to glm layers


Some weights of ChatGLMForConditionalGeneration were not initialized from the model checkpoint at chatglm-6b-int4 and are newly initialized: ['transformer.prefix_encoder.embedding.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


#### 最后来看看大语言模型微调后的回答。

In [12]:
response, history = model.chat(tokenizer, question, history=[])
print(f"question:{question}\nresponse:{response}")

question:朕的后宫里佳丽三千,你可知朕最宠幸的是谁?
response:皇上最宠幸的是谁？是皇后娘娘啊！她可是皇上的恩宠啊！
